# **LP3_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits."""

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

The variables for this mathematical optimization problem are:

1. X1: Number of units of product P1 produced in a week
2. X2: Number of units of product P2 produced in a week
3. X3: Number of units of product P3 produced in a week
4. Y1: Number of units of product P1 smelted at PRODA in a week
5. Y2: Number of units of product P2 smelted at PRODA in a week
6. Z1: Number of units of product P1 to subcontract for smelting in a week
7. Z2: Number of units of product P2 to subcontract for smelting in a week.


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
print(response2.choices[0].message.content)

Maximize profit = 1.50*(X1) - 0.30*(Y1) - 0.50*(Z1) - 0.20*(X1) -0.30*(X1) 
                + 1.80*(X2) - 0.50*(Y2) - 0.60*(Z2) - 0.10*(X2) - 0.20*(X2)
                + 1.97*(X3) - 0.40*(X3) - 0.27*(X3) - 0.20*(X3)


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [10]:
print(response3.choices[0].message.content)

1. Smelting Capacity Constraint: 6*Y1 + 10*Y2 + 8*X3 ≤ 8000
2. Mechanisation Capacity Constraint: 6*X1 + 3*X2 + 8*X3 ≤ 12000
3. Assembly and Packaging Capacity Constraint: 3*X1 + 2*X2 + 2*X3 ≤ 10000
4. Subcontracting Constraint for P1: Y1 + Z1 = X1
5. Subcontracting Constraint for P2: Y2 + Z2 = X2
6. Non-negativity Constraint: X1, X2, X3, Y1, Y2, Z1, Z2 ≥ 0


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [12]:
Markdown(response4.choices[0].message.content)

Solution:

Here is a Python script utilizing the Pyomo module to modelize the optimization problem:

```python
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define Sets
# N/A

# Define Parameters
# N/A

# Define Variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)
model.x3 = Var(domain=NonNegativeReals)
model.y1 = Var(domain=NonNegativeReals)
model.y2 = Var(domain=NonNegativeReals)
model.z1 = Var(domain=NonNegativeReals)
model.z2 = Var(domain=NonNegativeReals)

# Define Objective
model.profit = Objective(
    expr = 1.50*(model.x1) - 0.30*(model.y1) - 0.50*(model.z1) - 0.20*(model.x1) - 0.30*(model.x1)
            + 1.80*(model.x2) - 0.50*(model.y2) - 0.60*(model.z2) - 0.10*(model.x2) - 0.20*(model.x2)
            + 1.97*(model.x3) - 0.40*(model.x3) - 0.27*(model.x3) - 0.20*(model.x3),
    sense=maximize)

# Define Constraints
model.capacity_constraint1 = Constraint(
    expr = 6*model.y1 + 10*model.y2 + 8*model.x3 <= 8000)

model.capacity_constraint2 = Constraint(
    expr = 6*model.x1 + 3*model.x2 + 8*model.x3 <= 12000)

model.capacity_constraint3 = Constraint(
    expr = 3*model.x1 + 2*model.x2 + 2*model.x3 <= 10000)

model.subcontracting_constraint1 = Constraint(
    expr = model.y1 + model.z1 == model.x1)

model.subcontracting_constraint2 = Constraint(
    expr = model.y2 + model.z2 == model.x2)

# Solve
SolverFactory('glpk').solve(model).write()

# print the results
print("**** Results ****")
print("Profit = ", model.profit())
print("X1 = ", model.x1())
print("X2 = ", model.x2())
print("X3 = ", model.x3())
print("Y1 = ", model.y1())
print("Y2 = ", model.y2())
print("Z1 = ", model.z1())
print("Z2 = ", model.z2())
```

Note: This code uses the GLPK solver for solving the linear programming problem. If the GLPK solver is not installed, you can try using another solver that you have, by replacing `'glpk'` in `SolverFactory('glpk').solve(model).write()` with another solver. For example, for the CBC solver, you would replace `'glpk'` with `'cbc'`.

### **Run the code Generated by GPT4**

In [13]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define Sets
# N/A

# Define Parameters
# N/A

# Define Variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)
model.x3 = Var(domain=NonNegativeReals)
model.y1 = Var(domain=NonNegativeReals)
model.y2 = Var(domain=NonNegativeReals)
model.z1 = Var(domain=NonNegativeReals)
model.z2 = Var(domain=NonNegativeReals)

# Define Objective
model.profit = Objective(
    expr = 1.50*(model.x1) - 0.30*(model.y1) - 0.50*(model.z1) - 0.20*(model.x1) - 0.30*(model.x1)
            + 1.80*(model.x2) - 0.50*(model.y2) - 0.60*(model.z2) - 0.10*(model.x2) - 0.20*(model.x2)
            + 1.97*(model.x3) - 0.40*(model.x3) - 0.27*(model.x3) - 0.20*(model.x3),
    sense=maximize)

# Define Constraints
model.capacity_constraint1 = Constraint(
    expr = 6*model.y1 + 10*model.y2 + 8*model.x3 <= 8000)

model.capacity_constraint2 = Constraint(
    expr = 6*model.x1 + 3*model.x2 + 8*model.x3 <= 12000)

model.capacity_constraint3 = Constraint(
    expr = 3*model.x1 + 2*model.x2 + 2*model.x3 <= 10000)

model.subcontracting_constraint1 = Constraint(
    expr = model.y1 + model.z1 == model.x1)

model.subcontracting_constraint2 = Constraint(
    expr = model.y2 + model.z2 == model.x2)

# Solve
SolverFactory('glpk').solve(model).write()

# print the results
print("**** Results ****")
print("Profit = ", model.profit())
print("X1 = ", model.x1())
print("X2 = ", model.x2())
print("X3 = ", model.x3())
print("Y1 = ", model.y1())
print("Y2 = ", model.y2())
print("Z1 = ", model.z1())
print("Z2 = ", model.z2())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3680.0
  Upper bound: 3680.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 7
  Number of nonzeros: 15
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.004027843475341797
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**